## Examples using the FLP Database Connection Tools

### Import packages

In [1]:
import sys
import os
import pandas as pd
import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore", category=UserWarning, module='pandas.io.sql')

# Get the parent directory of the notebook (i.e., project root)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from Helpers.flp_database_connector import flp_database_connector

### Inputs

In [2]:
USERNAME = r"FIRSTLIGHTPOWER\CBrooks"
TABLE_NAME = "dbo.TestTable"
EXCEL_FILE_PATH = r"C:\Users\cbrooks\OneDrive - FIRSTLIGHTPOWER.COM\Quantitative Services - General\Data_Storage\Pricing_Data\ieso_rt_and_da_lmps_post_mrp.xlsx"

db_conn = flp_database_connector(USERNAME)

### Write data to table

In [6]:
# Upload Excel data
db_conn.upload_excel_to_quant_db(
    table_name=TABLE_NAME,
    excel_file=EXCEL_FILE_PATH,
    mode="overwrite"
    # skip_prompt defaults to False, keep it that way and prompt user to confirm overwrite
)

All previous data in dbo.TestTable cleared...
Upload complete: 17280 rows written to dbo.TestTable


In [7]:
# Upload Python data
df = pd.read_excel(EXCEL_FILE_PATH)
db_conn.upload_data_to_quant_db(
    table_name=TABLE_NAME,
    df=df,
    mode="overwrite"
    # skip_prompt defaults to False, keep it that way and prompt user to confirm overwrite
)

All previous data in dbo.TestTable cleared...
Upload complete: 17280 rows written to dbo.TestTable


### Read data from tables

In [10]:
# Query BURAPP007
ice_query = """
                select * from IceClearedPower where TradeDate = '2025-07-01' and Contract = 'NEP' and Strip = '2025-07-01'
            """
ice_data = db_conn.read_from_db("BURAPP007", "ICE", ice_query)
display(HTML(ice_data.to_html()))

c:\Users\cbrooks\OneDrive - FIRSTLIGHTPOWER.COM\Documents\Python\flp_database_connection_tools\Helpers\flp_database_connector.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TradeDate,Hub,Product,Strip,Contract,ContractType,Strike,SettlementPrice,NetChange,ExpirationDate,ProductID
0,2025-07-01,Nepool MH DA,Peak Futures (1 MW),2025-07-01,NEP,F,,71.15,0.29,2025-08-05,19687


In [11]:
# Query DataQuant01
quant_query = f"""
                select
                    *
                from
                    {TABLE_NAME}
                where
                    CAST(DateTime_HE AS DATE) = '2025-07-01'
                    and Pricing_Location = 'OZP'
                    and Market_Type = 'DA'
               """
quant_data = db_conn.read_from_db("DataQuant01", "", quant_query) # Only 1 database in DataQuant01, so leave that input blank
display(HTML(quant_data.to_html()))

c:\Users\cbrooks\OneDrive - FIRSTLIGHTPOWER.COM\Documents\Python\flp_database_connection_tools\Helpers\flp_database_connector.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,DateTime_HE,Pricing_Location,Wholesale_Market,Market_Type,LMP,Energy_Loss_Price,Energy_Congestion_Price,Unit,Currency,Interval_Width_s
0,2025-07-01 00:00:00,OZP,IESO,DA,43.51,-0.05,0.00,$/MWh,CAD,3600.0
1,2025-07-01 01:00:00,OZP,IESO,DA,68.36,-0.24,-0.13,$/MWh,CAD,3600.0
2,2025-07-01 02:00:00,OZP,IESO,DA,40.72,-0.06,-0.06,$/MWh,CAD,3600.0
3,2025-07-01 03:00:00,OZP,IESO,DA,34.06,0.04,-0.04,$/MWh,CAD,3600.0
4,2025-07-01 04:00:00,OZP,IESO,DA,30.22,0.05,-0.03,$/MWh,CAD,3600.0
5,2025-07-01 05:00:00,OZP,IESO,DA,29.07,0.05,-0.03,$/MWh,CAD,3600.0
6,2025-07-01 06:00:00,OZP,IESO,DA,35.77,0.00,0.00,$/MWh,CAD,3600.0
7,2025-07-01 07:00:00,OZP,IESO,DA,39.46,-0.02,0.00,$/MWh,CAD,3600.0
8,2025-07-01 08:00:00,OZP,IESO,DA,43.37,-0.12,0.00,$/MWh,CAD,3600.0
9,2025-07-01 09:00:00,OZP,IESO,DA,47.20,0.00,0.00,$/MWh,CAD,3600.0


### Create & Delete Tables

In [3]:
# Create a table
create_table_name = "dbo.TestTable1"
df = pd.read_excel(EXCEL_FILE_PATH)
db_conn.upload_data_to_quant_db(
        table_name=create_table_name, # Since this table doesn't exist, it will be created
        df=df,
        mode="create" # If we set mode to a different value, it would prompt us as to whether we actually wanted to create a new table
    )

Created new table: dbo.TestTable1
Upload complete: 17280 rows written to dbo.TestTable1


In [5]:
# Query data from the table
quant_query = f"""
                select
                    *
                from
                    {create_table_name}
                where
                    CAST(DateTime_HE AS DATE) = '2025-07-01'
                    and Pricing_Location = 'OZP'
                    and Market_Type = 'DA'
               """
quant_data = db_conn.read_from_db("DataQuant01", "", quant_query)
display(HTML(quant_data.to_html()))

c:\Users\cbrooks\OneDrive - FIRSTLIGHTPOWER.COM\Documents\Python\flp_database_connection_tools\Helpers\flp_database_connector.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,DateTime_HE,Pricing_Location,Wholesale_Market,Market_Type,LMP,Energy_Loss_Price,Energy_Congestion_Price,Unit,Currency,Interval_Width_s
0,2025-07-01 00:00:00,OZP,IESO,DA,43.51,-0.05,0.00,$/MWh,CAD,3600
1,2025-07-01 01:00:00,OZP,IESO,DA,68.36,-0.24,-0.13,$/MWh,CAD,3600
2,2025-07-01 02:00:00,OZP,IESO,DA,40.72,-0.06,-0.06,$/MWh,CAD,3600
3,2025-07-01 03:00:00,OZP,IESO,DA,34.06,0.04,-0.04,$/MWh,CAD,3600
4,2025-07-01 04:00:00,OZP,IESO,DA,30.22,0.05,-0.03,$/MWh,CAD,3600
5,2025-07-01 05:00:00,OZP,IESO,DA,29.07,0.05,-0.03,$/MWh,CAD,3600
6,2025-07-01 06:00:00,OZP,IESO,DA,35.77,0.00,0.00,$/MWh,CAD,3600
7,2025-07-01 07:00:00,OZP,IESO,DA,39.46,-0.02,0.00,$/MWh,CAD,3600
8,2025-07-01 08:00:00,OZP,IESO,DA,43.37,-0.12,0.00,$/MWh,CAD,3600
9,2025-07-01 09:00:00,OZP,IESO,DA,47.20,0.00,0.00,$/MWh,CAD,3600


In [6]:
# Delete the table
db_conn.delete_table_from_quant_db(create_table_name)

Table dbo.TestTable1 deleted successfully.
